<a href="https://colab.research.google.com/github/saisandeepponna/project-saisandeepponna-prasanna/blob/main/complete_emotions_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install keras-preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00


In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [ ]:
#Preparing google drive ot import the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/images.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [ ]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

fear completed
neutral completed
angry completed
disgust completed
surprise completed
happy completed
sad completed


In [ ]:
print(train)

                             image label
0      images/train/fear/17651.jpg  fear
1      images/train/fear/33557.jpg  fear
2      images/train/fear/13691.jpg  fear
3       images/train/fear/2097.jpg  fear
4      images/train/fear/24787.jpg  fear
...                            ...   ...
28816   images/train/sad/19542.jpg   sad
28817   images/train/sad/33754.jpg   sad
28818   images/train/sad/10671.jpg   sad
28819    images/train/sad/7864.jpg   sad
28820   images/train/sad/22488.jpg   sad

[28821 rows x 2 columns]


In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

fear completed
neutral completed
angry completed
disgust completed
surprise completed
happy completed
sad completed


In [ ]:
print(test)
print(test['image'])

                           image label
0     images/test/fear/23083.jpg  fear
1     images/test/fear/12255.jpg  fear
2      images/test/fear/2391.jpg  fear
3     images/test/fear/11137.jpg  fear
4     images/test/fear/13855.jpg  fear
...                          ...   ...
7061   images/test/sad/20301.jpg   sad
7062    images/test/sad/6930.jpg   sad
7063   images/test/sad/17763.jpg   sad
7064   images/test/sad/25055.jpg   sad
7065   images/test/sad/20072.jpg   sad

[7066 rows x 2 columns]
0       images/test/fear/23083.jpg
1       images/test/fear/12255.jpg
2        images/test/fear/2391.jpg
3       images/test/fear/11137.jpg
4       images/test/fear/13855.jpg
                   ...            
7061     images/test/sad/20301.jpg
7062      images/test/sad/6930.jpg
7063     images/test/sad/17763.jpg
7064     images/test/sad/25055.jpg
7065     images/test/sad/20072.jpg
Name: image, Length: 7066, dtype: object


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


In [ ]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [ ]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 250, validation_data = (x_test,y_test))

Epoch 1/250
226/226 [==============================] - 26s 54ms/step - loss: 1.8248 - accuracy: 0.2415 - val_loss: 1.8084 - val_accuracy: 0.2583
Epoch 2/250
226/226 [==============================] - 12s 52ms/step - loss: 1.7944 - accuracy: 0.2557 - val_loss: 1.7221 - val_accuracy: 0.2969
Epoch 3/250
226/226 [==============================] - 11s 49ms/step - loss: 1.6865 - accuracy: 0.3137 - val_loss: 1.5443 - val_accuracy: 0.4052
Epoch 4/250
226/226 [==============================] - 11s 50ms/step - loss: 1.5486 - accuracy: 0.3950 - val_loss: 1.4738 - val_accuracy: 0.4517
Epoch 5/250
226/226 [==============================] - 11s 49ms/step - loss: 1.4656 - accuracy: 0.4330 - val_loss: 1.3710 - val_accuracy: 0.4690
Epoch 6/250
226/226 [==============================] - 11s 50ms/step - loss: 1.4047 - accuracy: 0.4574 - val_loss: 1.2797 - val_accuracy: 0.5105
Epoch 7/250
226/226 [==============================] - 12s 52ms/step - loss: 1.3656 - accuracy: 0.4734 - val_loss: 1.2468 - val_ac

In [ ]:
model_json = model.to_json()
with open("emotiondetectormore.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetectormore.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0


In [ ]:
image = '/content/images/train/fear/10012.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of fear


/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


1/1 [==============================] - 0s 312ms/step
model prediction is  fear
